In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [28]:
CFG = {
    "csv_path":      "mumbai_weekly_tariffs.csv",
    "battery_kwh":   13.5,
    "battery_kw":    5.0,
    "eta_round":     0.95,
    "soc_initial":   0.3,
    "house_load_kw": 0.8,
    "plot_hours":    168,
    # optional plotting switch – keep False until matplotlib is clean
    "PLOTTING":      False,
}

In [23]:
def read_mumbai_csv(path: str) -> pd.DataFrame:
    """Return hourly DataFrame with Asia/Kolkata DatetimeIndex."""
    df = pd.read_csv(path)
    df["date"] = pd.to_datetime(df["Date"])
    df["start_hour"] = df["Time_Slot"].str.split("-").str[0].str[:2].astype(int)
    df["timestamp"] = df["date"] + pd.to_timedelta(df["start_hour"], unit="h")

    hourly = []
    for _, r in df.iterrows():
        idx = pd.date_range(r["timestamp"], freq="H", periods=6, tz="Asia/Kolkata")
        hourly.append(pd.DataFrame({"timestamp": idx, "tariff": r["Tariff_(₹/kWh)"]}))
    dfh = pd.concat(hourly).drop_duplicates("timestamp").sort_values("timestamp")
    return dfh.set_index("timestamp")

In [24]:
def plan_hour(tariff: float, load_kw: float,
              soc_prev: float, bat_kwh: float, bat_kw: float, eta: float,
              threshold: float) -> dict:
    """
    Threshold = cheap vs expensive cut-off (₹/kWh).
    Greedy one-step bill minimiser.
    """
    if tariff >= threshold:          # EXPENSIVE → discharge
        max_discharge = min(bat_kw, soc_prev * bat_kwh)
        discharge_kw  = min(max_discharge, load_kw / eta)
        grid_kw   = max(0.0, load_kw - discharge_kw * eta)
        battery_kw = discharge_kw
        soc_new    = soc_prev - discharge_kw / bat_kwh
        mode       = "discharge"
    else:                            # CHEAP → charge
        max_charge = min(bat_kw, (1 - soc_prev) * bat_kwh)
        battery_kw = -max_charge
        grid_kw    = load_kw + max_charge
        soc_new    = soc_prev + max_charge * eta / bat_kwh
        mode       = "charge"

    return {"grid_kw": grid_kw,
            "battery_kw": battery_kw,
            "soc_new": np.clip(soc_new, 0, 1),
            "mode": mode}

In [25]:
def simulate(df_future: pd.DataFrame, cfg: dict) -> pd.DataFrame:
    records, soc = [], cfg["soc_initial"]
    threshold = df_future["tariff"].mean()   # <- data-driven, no placeholder
    print(f"Optimal threshold (future mean): {threshold:.2f} ₹/kWh")

    for ts, row in df_future.iterrows():
        res = plan_hour(row["tariff"], cfg["house_load_kw"],
                        soc, cfg["battery_kwh"], cfg["battery_kw"], cfg["eta_round"],
                        threshold)
        records.append({"timestamp": ts,
                        "tariff": row["tariff"],
                        **res})
        soc = res["soc_new"]
    return pd.DataFrame(records).set_index("timestamp")

In [26]:
df_all    = read_mumbai_csv(CFG["csv_path"])
now       = pd.Timestamp.now(tz="Asia/Kolkata").floor("H")
df_future = df_all[df_all.index >= now]
if df_future.empty:
    raise RuntimeError("CSV has no future records – cannot schedule.")

schedule = simulate(df_future, CFG)
print("Simulation finished – first 5 rows:")
display(schedule.head())

Optimal threshold (future mean): 6.19 ₹/kWh
Simulation finished – first 5 rows:


C:\Users\HP\AppData\Local\Temp\ipykernel_42196\2583057915.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  idx = pd.date_range(r["timestamp"], freq="H", periods=6, tz="Asia/Kolkata")
C:\Users\HP\AppData\Local\Temp\ipykernel_42196\4257710589.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  now       = pd.Timestamp.now(tz="Asia/Kolkata").floor("H")


,tariff,grid_kw,battery_kw,soc_new,mode
timestamp,,,,,
2025-10-28 20:00:00+05:30,7.5,0.0,0.842105,0.237622,discharge
2025-10-28 21:00:00+05:30,7.5,0.0,0.842105,0.175244,discharge
2025-10-28 22:00:00+05:30,7.5,0.0,0.842105,0.112865,discharge
2025-10-28 23:00:00+05:30,7.5,0.0,0.842105,0.050487,discharge
2025-10-29 00:00:00+05:30,5.0,5.8,-5.000000,0.402339,charge


In [29]:
if CFG["PLOTTING"]:
    import matplotlib.pyplot as plt          # import only here, only once
    def plot_tariff(sched: pd.DataFrame, horizon: int):
        sub = sched.head(horizon)
        plt.figure(figsize=(14, 4))
        plt.plot(sub.index, sub["tariff"], label="Hourly tariff")
        plt.ylabel("Tariff (₹/kWh)")
        plt.title("Electricity tariff")
        plt.legend()
        plt.tight_layout()
        plt.show()
    plot_tariff(schedule, CFG["plot_hours"])

In [8]:
if CFG["PLOTTING"]:
    def plot_battery_delivery(sched: pd.DataFrame, horizon: int):
        sub = sched.head(horizon)
        plt.figure(figsize=(14, 4))
        plt.step(sub.index, sub["battery_kw"], where="post", color="C1")
        plt.axhline(0, color="grey", lw=0.8)
        plt.ylabel("Battery power (kW)")
        plt.title("Optimal battery delivery power (+ = discharge)")
        plt.tight_layout()
        plt.show()
    plot_battery_delivery(schedule, CFG["plot_hours"])

In [9]:
schedule.to_csv("optimal_battery_power_schedule.csv")
print("Saved →", Path("optimal_battery_power_schedule.csv").resolve())

Saved → C:\Users\HP\Documents\Codes\Enervia\optimal_battery_power_schedule.csv


In [ ]:
# ----------------------------------------------------------
# Cell 10 – cost vectors + % saving per kWh per day
# ----------------------------------------------------------
schedule["grid_only_cost"] = schedule["tariff"] * CFG["house_load_kw"]          # ₹/h
schedule["our_cost"]       = schedule["tariff"] * schedule["grid_kw"]           # ₹/h
schedule["saving_per_kwh"] = schedule["grid_only_cost"] - schedule["our_cost"]  # ₹/h

# percentage saved per kWh each hour
schedule["pct_saved"] = (schedule["saving_per_kwh"] /
                         schedule["grid_only_cost"].replace(0, np.nan)) * 100   # %

# daily average
daily_avg_pct = (schedule["pct_saved"]
                 .groupby(schedule.index.date)
                 .mean()
                 .mean())          # mean-of-daily-means

# cumulative money lines (for later plots)
schedule["cum_grid_only"] = schedule["grid_only_cost"].cumsum()
schedule["cum_our"]       = schedule["our_cost"].cumsum()

print("Total money saved over horizon:",
      f"{schedule['saving_per_kwh'].sum():.2f} ₹")
print("Average saving per kWh:",
      f"{schedule['saving_per_kwh'].mean():.3f} ₹/kWh")

display(schedule[["tariff", "grid_kw", "battery_kw", "soc_new",
                  "grid_only_cost", "our_cost", "saving_per_kwh", "pct_saved"]].head())

Total money saved over horizon: 7.73 ₹
Average saving per kWh: 0.080 ₹/kWh
Average percentage saved per kWh per day: 38.5 %


,tariff,grid_kw,battery_kw,soc_new,grid_only_cost,our_cost,saving_per_kwh,pct_saved
timestamp,,,,,,,,
2025-10-28 19:00:00+05:30,7.5,0.0000,0.842105,0.237622,6.0,0.00000,6.00000,100.0000
2025-10-28 20:00:00+05:30,7.5,0.0000,0.842105,0.175244,6.0,0.00000,6.00000,100.0000
2025-10-28 21:00:00+05:30,7.5,0.0000,0.842105,0.112865,6.0,0.00000,6.00000,100.0000
2025-10-28 22:00:00+05:30,7.5,0.0000,0.842105,0.050487,6.0,0.00000,6.00000,100.0000
2025-10-28 23:00:00+05:30,7.5,0.1525,0.681579,0.000000,6.0,1.14375,4.85625,80.9375


In [16]:
if CFG["PLOTTING"]:
    sub = schedule.head(CFG["plot_hours"])
    fig, ax = plt.subplots(figsize=(14, 4))
    ax.step(sub.index, sub["battery_kw"], where="post", label="Battery power (+ discharge)", color="C1")
    ax.axhline(0, color="grey", lw=0.8)
    ax.set_ylabel("kW")
    ax.set_title("Battery deployment schedule")
    ax.legend()
    plt.tight_layout()
    plt.show()

In [17]:
if CFG["PLOTTING"]:
    sub = schedule.head(CFG["plot_hours"])
    plt.figure(figsize=(14, 4))
    plt.plot(sub.index, sub["cum_grid_only"], label="Grid-only cumulative cost", color="black")
    plt.plot(sub.index, sub["cum_our"],       label="Battery-assisted cumulative cost", color="green")
    plt.ylabel("Cumulative cost (₹)")
    plt.title("Money spent over time – both scenarios")
    plt.legend()
    plt.tight_layout()
    plt.show()